In [1]:
using PorousMaterials # Pkg.clone("https://github.com/SimonEnsemble/PorousMaterials.jl", "v0.1.1")
using CSV
using DataFrames
using DelimitedFiles
using LinearAlgebra
using Printf
using JLD2

In [2]:
cages = readdlm("all_cages/all_cages.txt")
cages[1]

"A11"

In [3]:
aligned_and_centered_cage(cage::AbstractString) = read_xyz("all_cages/" * cage * "_final_alignment.xyz")

aligned_and_centered_cage (generic function with 1 method)

In [4]:
xe = Molecule("Xe")

cutoffradius = 14.0 # vdw cutoff radius
ljff = LJForceField("UFF.csv", cutoffradius=cutoffradius, mixing_rules="geometric");

In [5]:
atoms, x = aligned_and_centered_cage("A11")
max_length = ceil(2.0 * maximum(abs.(x)))

center = [max_length / 2.0 for s = 1:3]
box = Box(max_length, max_length, max_length, π/2, π/2, π/2)

nb_grid_pts = ceil(Int64, 10 * max_length)

x_grid_pts = collect(range(0, stop=max_length, length=nb_grid_pts)) .- max_length / 2.0

@printf("Number of grid points: [%d, %d, %d]\n", nb_grid_pts, nb_grid_pts, nb_grid_pts)
# assert snapshot box is big enough
@assert(all(x .< maximum(x_grid_pts)) && all(x .> minimum(x_grid_pts)),
        "atoms outside snapshot!")

Number of grid points: [180, 180, 180]


In [7]:
accessible_xe = trues(length(cages))

for (c, cage) in enumerate(cages)
    # read in aligned and centered cage
    @printf("\nWorking on cage: %s\n", cage)
    atoms, x = aligned_and_centered_cage(cage)
    max_length = ceil(2.0 * maximum(abs.(x)))
    #x_span = 2.0 * maximum(abs.(x))
    
    center = [max_length / 2.0 for s = 1:3]
    box = Box(max_length, max_length, max_length, π/2, π/2, π/2)
    
    nb_grid_pts = ceil(Int, 5 * max_length)
    x_grid_pts = collect(range(0, stop=max_length, length=nb_grid_pts)) .- max_length / 2.0
    
    @printf("Number of grid points: [%d, %d, %d]\n", nb_grid_pts, nb_grid_pts, nb_grid_pts)

    # assert snapshot box is big enough
    @assert(all(x .< maximum(x_grid_pts)) && all(x .> minimum(x_grid_pts)),
        "atoms outside snapshot!")
    
    # convert cage to a list of LJSphere's for PorousMaterials.jl
    ljspheres = Atoms(atoms, x)
    
    # preallocate grid; choose origin here solely for visualization
    # when we write to a .cube file. the origin passed to `Grid`
    # is so the cage is centered at the origin in the viz. (checked)
    grid_xe = Grid(box, (nb_grid_pts, nb_grid_pts, nb_grid_pts), 
                zeros(Float64, nb_grid_pts, nb_grid_pts, nb_grid_pts), 
                :energy, -center)
        
    for i = 1:nb_grid_pts, j = 1:nb_grid_pts, k = 1:nb_grid_pts
        # what grid pt is here inside the loop?
        x_grid_pt = [x_grid_pts[i], x_grid_pts[j], x_grid_pts[k]]
        #println(x_grid_pt)
        
        # put helium at this grid pt
        translate_to!(xe, x_grid_pt)
        
        # compute potential energy of He adsorbate here
        energy_xe = vdw_energy_no_PBC(xe, ljspheres, ljff)
        
        # if framework atom overlaps with cage atom... energy > k T
        grid_xe.data[i, j, k] = energy_xe
        
    end
    # 15 kT at room temperature (T = 298 K)
    segmented_grid_xe = PorousMaterials._segment_grid(grid_xe, 15.0*298.0, false)
    n_center = ceil(Int, nb_grid_pts/2)
    if length(unique(segmented_grid_xe.data)) > 2 && segmented_grid_xe.data[1, 1, 1] != segmented_grid_xe.data[n_center, n_center, n_center]
        @printf("\tXe\n\tOuter Segment: %d\tInner Segment: %d\t Number of segments: %d\n", 
                segmented_grid_xe.data[1,1,1], segmented_grid_xe.data[n_center,n_center,n_center],length(unique(segmented_grid_xe.data)))
        accessible_xe[c] = false
    end
    if accessible_xe[c]
        @printf("\t Same segment inside and outside\n\t Number of segments: %d (Xe)\n", 
                length(unique(segmented_grid_xe.data)))
    end
end

open("inaccessible_cages.txt", "w") do f
    for cage in cages[.!accessible_xe]
        @printf(f, "%s\n", cage)
    end
end
@printf("See inaccessible_cages.txt for cages that are inaccessible towards Xenon\n")


Working on cage: A11
Number of grid points: [90, 90, 90]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: B11
Number of grid points: [95, 95, 95]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: B13
Number of grid points: [105, 105, 105]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: B15
Number of grid points: [125, 125, 125]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: B18
Number of grid points: [135, 135, 135]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: B1
Number of grid points: [70, 70, 70]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: B23
Number of grid points: [110, 110, 110]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: B24
Number of grid points: [120, 120, 120]
	 Same segment inside and outside
	 Number of segments: 2 (Xe)

Working on cage: 